# Illustration de l'interblocage en Python
*Licence Creative Commons BY-SA 4.0 — Olivier Lecluse — 2020*

## Activité préliminaire

Commençons par une simple fonction qui doit incrémenter 100 fois une variable compteur :

In [ ]:
from time import sleep

compteur = 0 # Variable globale
limite = 100

def calcul():
    global compteur
    for i in range(limite):
        temp = compteur
        # simule un traitement nécessitant des calculs
        sleep(0.000000001)
        compteur = temp + 1

*Remarque*  : la variable ```compteur``` est définie comme une variable globale dans la fonction (mot clef global) et donc la modification de cette varibale au sein de la fonction se répercute pour l'ensemble du programme.

In [ ]:
compteur = 0
calcul()

In [ ]:
print(compteur)

Sans surprise, la valeur de la variable `compteur` vaut 100. Vous remarquerez la manière dont l'incrémentation du compteur a été volontairement complexifiée. La raison va apparaître maintenant car nous allons maintenant lancer 4 processus légers (appelés **threads**) qui vont tous les 4 exécuter la fonction calcul.

On pourrait imaginer que puisque calcul est exécuté 4 fois le résultat final sera 400. En réalité, il n'en est rien :

In [ ]:
from threading import Thread

In [ ]:
compteur = 0
# création de 4 threads pour exécuter la fonction calcul :
p1 = Thread(target=calcul)
p2 = Thread(target=calcul)
p3 = Thread(target=calcul)
p4 = Thread(target=calcul)
# démarrage des 4 threads :
p1.start()      # Lance calcul dans un processus léger à part
p2.start()      # Lance calcul dans un 2ème processus léger à part
p3.start()      # Lance calcul dans un 3ème processus léger à part
p4.start()      # Lance calcul dans un 4ème processus léger à part

In [ ]:
print(compteur)

## Threads are evil, don't use them !

En réévaluant plusieurs fois l'exécution du calcul parallèle, on s'aperçoit que :
- le résultat n'est pas 400 !
- d'une exécution à l'autre le résultat est différent !!

Voila qui est très perturbant : Un même programme - très simple - exécuté plusieurs fois qui ne donne pas le même résultat. Bienvenue dans le monde infernal des threads. Cela explique la réticence de certains à les utiliser : **Threads are evil, don't use them !** lit-on souvent sur les forums de développeurs...

Et pourtant ils sont nécessaires à bon nombre de tâches. Sans threads, pas de serveurs webs capables d'encaisser des millions de requêtes à la seconde !

## Explication

Il n'y a rien d'illogique ou d'aléatoire dans le fonctionnement de notre programme. Il faut simplement habituer notre esprit à l'exécution en parallèle :
- 4 processus (appelons les P1, P2, P3 et P4) exécutent la fonction calcul simultanément. Celle-ci utilise une **variable globale** ```compteur``` qui sera donc modifiée par chacun de ces processus et une **variable locale** ```temp``` qui sera spécifique à chacun de nos processus. Nous la désignerons par temp(P1) temp(P2) etc... 
Un scénario possible est le suivant : Imaginons au départ que compteur vaille 10.
1. P1 sauvegarde compteur dans temp(P1) --> temp(P1) vaut 10
2. P2 sauvegarde compteur dans temp(P2) --> temp(P2) vaut 10
3. P3 sauvegarde compteur dans temp(P3) --> temp(P3) vaut 10
4. P1 et P2 incrémentent temp et sauvegardent la réponse dans compteur --> compteur vaut 11
5. P4 sauvegarde compteur dans temp(P4) --> temp(P4) vaut 11
6. P3 et P4 incrémentent temp et sauvegardent la réponse dans compteur

Au final, compteur a été incrémenté 4 fois mais de fait de l'exécution en parallèle compteur ne vaut pas 14 mais 12 ! cela explique que notre compteur au final ne vaut pas 400 car sa sauvegarde dans des variables temporaires fait que la plupart des incrémentations ne sont pas prises en compte.

Quand au résultat apparemment aléatoire, il est dû au scénario d'exécution. On a cité un au hasard mais bien d'autres sont possibles qui mèneraient à des résultats différents. Un problème avec les threads est que l'on ne maîtrise pas du tout l'ordre dans lequel ils sont exécutés. Une fois lancés, chacun vit sa vie...

Vous voila j'espère convaincus que les threads sont réellement diaboliques !

## Fiabiliser l'algorithme

Il est possible d'éviter que nos threads interfèrent les uns avec les autres : il suffit de s'assurer que la partie centrale qui incrémente notre compteur ne soit pas exécutée par 2 threads à la fois. Pour ce faire, on introduit la notion de **verrou** : un verrou peut être vu comme un témoin qui passe de thread en thread. Seul celui qui possède ce témoin peut exécuter l'incrémentation du compteur, les autres doivent attendre leur tour. 

Ce verrou peut-être vu comme la ressource nécessaire à l'exécution d'une tâche pour un processus (voir cours). Tant que le processus ne possède pas le verrou (la ressource), il est dans un état bloqué. Lorsqu'il acquiert le verrou, il passe à l'état prêt et peut reprendre son exécution en passant à l'état élu. Une différence toutefois, avec les threads ce n'est pas le système d'exploitation qui gère l'ordonnancement, mais celui-ci vient directement de la programmation.

Nous allons donc légèrement modifier notre fonction calcul afin d'utiliser la partie critique de notre algorithme, à savoir l'incrémentation du compteur. 
Le module threading propose un objet Lock() destiné à cette tâche. Deux méthodes seront utilisées ici :
- verrou.acquire() : attend que le verrou soit libéré pour se l'accaparer et l'activer
- verrou.release() : libère le verrou

In [ ]:
from threading import Lock

In [ ]:
verrou = Lock()

def calcul1():
    global compteur
    for i in range(limite):
        verrou.acquire()
        # Début de la section critique
        temp = compteur
        sleep(0.000000001)
        compteur = temp + 1
        # Fin de la section critique
        verrou.release()

Grâce à ce système, il est impossible que la partie critique du code soit exécutée simultanément par deux threads. On est donc assuré qu'il n'y aura pas d'interaction entre les threads.

Au final on constate bien que le le compteur vaut 400, ce qui est le résultat attendu (voir cellules ci-dessous).

En mémorisant les threads créés dans une liste, on pourra constater qu'ils se sont tous bien terminés.

In [ ]:
compteur = 0

mes_threads = []
for i in range(4):
    p = Thread(target=calcul1)
    mes_threads.append(p)
    p.start()

In [ ]:
print(compteur)

Dans quel état se trouve le 1er thread ? (reponse ci-dessous, assez explicite, même sans doc)

In [ ]:
print(mes_threads[0])
print(mes_threads[0].is_alive())

In [ ]:
mes_threads # on vérifie qu'ils sont bien tous terminés

## Et l'interblocage ?

La situation d'interblocage ne peut pas arriver dans le scénario précédent car nous n'avons qu'un verrou donc *une seule resssource* convoitée par 4 threads : celle-ci va passer de threads en threads sans conflits. **Pour faire apparaître l'interblocage, nous avons besoin de plusieurs verrous afin de créer une attente circulaire**.

Pour reproduire l'exemple vu en cours, nous allons  créer 2 verrous (V1 et V2) qui simulent les deux ressources (R1 et R2), et manipuler 2 threads (P1 et P2). La fonction calcul2 va utiliser deux verrous. 

    Le Thread P1 va donc réserver V1 puis V2
    Le Thread P2 va de son côté réserver V2 puis V1

Ces verrous seront libérés à la fin... sauf si le thread est en attente pour acquerir un verrou ! Et voilà une situation d'interblocage qui s'annonce...

In [ ]:
def calcul2(verrou1, verrou2):
    global compteur
    for i in range(limite):
        verrou1.acquire()
        # Début de la section critique 1 
        temp = compteur
        sleep(0.000000001)
        # Début de la section critique 2
        verrou2.acquire()
        compteur = temp + 1
        # Début de la section critique 2
        verrou2.release()
        # Fin de la section critique 1
        verrou1.release()

In [ ]:
compteur = 0

# création des 2 verrous :
v1, v2 = Lock(), Lock()

# création des 2 threads :
p1 = Thread(target=calcul2, args=[v1, v2])
p2 = Thread(target=calcul2, args=[v2, v1])

# démarrage des  threads :
p1.start()
#sleep(2)
p2.start()

Où en est le compteur ? Dans quel état sont les threads ?

In [ ]:
print(compteur)
p1, p2

On constate que tous les threads sont actifs et que le compteur ne s'incrémente pas. Nous sommes en situation d'interblocage !

Il n'y a pas d'autre choix que de tuer le processus Python et les threads afférents.

# Exercice :
Obervez la fonction suivante et son exécution simultanée par 2 threads.

In [ ]:
def ecrire(lettre):
    for i in range(20):
        print(lettre, end='')
        sleep(.000001)

In [ ]:
p1 = Thread(target=ecrire, args=['a'])
p2 = Thread(target=ecrire, args=['b'])
p1.start()
p2.start()

Reécrire la fonction ecrire en utilisant un verrou pour que les 2 threads s'éxécutent à la suite.

On veut voir 'affichage :
    
    aaaaaaaaaaaaaaaaaabbbbbbbbbbbbbbbbbbb.

In [ ]:
### A VOUS DE JOUER
